In [67]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score #Confusion matrix 수업 때 진행할 예정 
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
import pandas as pd
import seaborn as sns
df = sns.load_dataset("titanic")

In [68]:
df.isnull().sum()
# deck 이라는 변수 드랍 시키기
df=df.drop("deck", axis=1)
# 결측치가 들어있는 행 삭제 시키기
df=df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 0 to 890
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     712 non-null    int64   
 1   pclass       712 non-null    int64   
 2   sex          712 non-null    object  
 3   age          712 non-null    float64 
 4   sibsp        712 non-null    int64   
 5   parch        712 non-null    int64   
 6   fare         712 non-null    float64 
 7   embarked     712 non-null    object  
 8   class        712 non-null    category
 9   who          712 non-null    object  
 10  adult_male   712 non-null    bool    
 11  embark_town  712 non-null    object  
 12  alive        712 non-null    object  
 13  alone        712 non-null    bool    
dtypes: bool(2), category(1), float64(2), int64(4), object(5)
memory usage: 69.0+ KB


In [70]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,Southampton,no,True


In [71]:
df.isnull().sum()

survived       0
pclass         0
sex            0
age            0
sibsp          0
parch          0
fare           0
embarked       0
class          0
who            0
adult_male     0
embark_town    0
alive          0
alone          0
dtype: int64

# alive, adult_male, who , embark_town은 크게 의미없는 데이터라 생각했음

In [72]:
df=df.drop('alive', axis=1)
df=df.drop('adult_male', axis=1)
df=df.drop("who", axis=1)
df=df.drop("embark_town", axis=1)

In [73]:
from sklearn import preprocessing

def encode_features(dataDF):
    features=['sex','embarked','class']
    for feature in features:
        le = preprocessing.LabelEncoder()
        le = le.fit(dataDF[feature])
        dataDF[feature] = le.transform(dataDF[feature])
    return dataDF
df = encode_features(df)
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,alone
0,0,3,1,22.0,1,0,7.2500,2,2,False
1,1,1,0,38.0,1,0,71.2833,0,0,False
2,1,3,0,26.0,0,0,7.9250,2,2,True
3,1,1,0,35.0,1,0,53.1000,2,0,False
4,0,3,1,35.0,0,0,8.0500,2,2,True


In [74]:
def change_to_index(columns):
    tmp = {string : i for i,string in enumerate(df[columns].unique())}
    df[columns] = df[columns].map(tmp)
    print(tmp)

In [75]:
# 종속변수 이름 설정하기
label_name='survived'
# 독립변수 이름 리스트 설정하기
feature_names=df.columns.tolist()
feature_names.remove(label_name)
feature_names
# 각 X와 y 만들어주기
X=df[feature_names]
y=df[label_name]
X.shape, y.shape
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=111)

In [76]:
model = DecisionTreeClassifier()
model_sp1 = DecisionTreeClassifier(random_state=111, max_depth=3)
model.fit(X_train, y_train) #모델 학습
model_sp1.fit(X_train, y_train) #모델 학습
pred_train = model.predict(X_train) #train 예측
pred_test = model.predict(X_test) # test예측

pred_train_sp = model_sp1.predict(X_train) #train 예측
pred_test_sp = model_sp1.predict(X_test) # test예측

In [77]:
print(accuracy_score(y_train, pred_train))
print(accuracy_score(y_test, pred_test))

0.9859402460456942
0.8041958041958042


In [78]:
print(accuracy_score(y_train, pred_train_sp))
print(accuracy_score(y_test, pred_test_sp))

0.8224956063268892
0.8111888111888111


In [85]:
from sklearn.model_selection import KFold

kfold= KFold(n_splits=5) #데이터셋을 분리하는 것

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=111)
cv_accuracy_train=[]
cv_accuracy_test=[]
kf_model = DecisionTreeClassifier(random_state=111, max_depth=3)

In [86]:
kfold

KFold(n_splits=5, random_state=None, shuffle=False)

In [87]:
n_iter = 0 #5번 진행하니 초깃값 설정

for train_idx, test_idx in kfold.split(X):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
    #학습을 위해서 dt배우기
    kf_model.fit(X_train, y_train)
    #예측
    kf_pred_train = kf_model.predict(X_train)
    df_pred_test = kf_model.predict(X_test)
    
    #정확도를 5번 측정하면서 값에 넣을 것이니
    n_iter +=1
    accuracy_train = np.round(accuracy_score(y_train, kf_pred_train),4)
    accuracy_test = np.round(accuracy_score(y_test, df_pred_test),4)
    
    print('\n{} 번 train 교차 검증 정확도 :{} ,test 교차 검증 정확도 :{} '.format(n_iter,accuracy_train, accuracy_test))
    
    cv_accuracy_train.append(accuracy_train)
    cv_accuracy_test.append(accuracy_test)
    
print('train 평균 정확도',np.mean(cv_accuracy_train))
print('test 평균 정확도',np.mean(cv_accuracy_test))


1 번 train 교차 검증 정확도 :0.8172 ,test 교차 검증 정확도 :0.8182 

2 번 train 교차 검증 정확도 :0.8102 ,test 교차 검증 정확도 :0.7972 

3 번 train 교차 검증 정확도 :0.8281 ,test 교차 검증 정확도 :0.7887 

4 번 train 교차 검증 정확도 :0.8333 ,test 교차 검증 정확도 :0.7676 

5 번 train 교차 검증 정확도 :0.807 ,test 교차 검증 정확도 :0.8028 
train 평균 정확도 0.8191600000000001
test 평균 정확도 0.7948999999999999


In [88]:
from sklearn.model_selection import StratifiedKFold

skf_iris = StratifiedKFold(n_splits=3)
cnt_iter=0

In [89]:
skf_iris

StratifiedKFold(n_splits=3, random_state=None, shuffle=False)

In [90]:
n_iter=0

skf_cv_accuracy_train = []
skf_cv_accuracy_test = []
skf_model = DecisionTreeClassifier(random_state=111, max_depth=3)

for train_idx, test_idx in skf_iris.split(X, y):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
    #학습을 위해서 dt배우기
    skf_model.fit(X_train, y_train)
    #예측
    skf_pred_train = skf_model.predict(X_train)
    skf_pred_test = skf_model.predict(X_test)
    
    #정확도를 5번 측정하면서 값에 넣을 것이니
    n_iter +=1
    accuracy_train = np.round(accuracy_score(y_train, skf_pred_train),4)
    accuracy_test = np.round(accuracy_score(y_test, skf_pred_test),4)
    
    print('\n{} 번 train 교차 검증 정확도 :{} ,test 교차 검증 정확도 :{} '.format(n_iter,accuracy_train, accuracy_test))
    
    skf_cv_accuracy_train.append(accuracy_train)
    skf_cv_accuracy_test.append(accuracy_test)
    
print('train 평균 정확도',np.mean(skf_cv_accuracy_train))
print('test 평균 정확도',np.mean(skf_cv_accuracy_test))


1 번 train 교차 검증 정확도 :0.8291 ,test 교차 검증 정확도 :0.7521 

2 번 train 교차 검증 정확도 :0.8232 ,test 교차 검증 정확도 :0.7932 

3 번 train 교차 검증 정확도 :0.8316 ,test 교차 검증 정확도 :0.827 
train 평균 정확도 0.8279666666666666
test 평균 정확도 0.7907666666666667
